# ENTREGABLE 4

# INSTRUCCIONES

Utilizar el archivo CSV (`dataset_banco_clean.csv`) con 45189 filas y 17 columnas y aplicar las técnicas de normalización del entregable 3.

In [3]:
# Módulos necesarios
!pip install scikit-learn imblearn
!pip install xgboost
!pip install lightgbm
!pip install catboost

import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.stats import expon, reciprocal
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Clasificadores
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier, StackingClassifier, HistGradientBoostingClassifier
# from sklearn.svm import SVC
# from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import RidgeClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


In [4]:
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [5]:

iris=load_iris()

In [6]:
data = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Añadir la columna de etiquetas al DataFrame
data['species'] = iris.target
data['species'] = data['species'].apply(lambda x: iris.target_names[x])
data.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [7]:
data.isna().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
species              0
dtype: int64

# Objetivo

Generar un model de clasificación capaz de predecir la clase de flor en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Generar split de los datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

In [8]:
from sklearn.preprocessing import LabelEncoder

label_mappings={}

le = LabelEncoder()

data['species'] = le.fit_transform(data['species'])
# Almacenar las clases y sus etiquetas correspondientes
label_mappings['species'] = dict(zip(le.classes_, le.transform(le.classes_)))

print("\nAsignaciones de Label Encoder para cada columna:")
for column, mappings in label_mappings.items():
    print(f"{column}: {mappings}")


Asignaciones de Label Encoder para cada columna:
species: {'setosa': 0, 'versicolor': 1, 'virginica': 2}


In [9]:
# Definimos el set de características y el set de la variable a predecir Y
X = data.drop(columns='species')
Y = data['species']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.15)
print("Entrenamiento:", X_train.shape, y_train.shape)
print("Validación:", X_val.shape, y_val.shape)
print("Prueba:", X_test.shape, y_test.shape)

Entrenamiento: (127, 4) (127,)
Validación: (23, 4) (23,)
Prueba: (23, 4) (23,)


In [29]:
modelos_mapeados = {
    "Regresión Logística": LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=200),
    "K-Nearest Neighbors (KNN)": KNeighborsClassifier(),
    "Árboles de Decisión": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Máquinas de Vectores de Soporte (SVM)": SVC(probability=True),
    "AdaBoost": AdaBoostClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    "Clasificador de Vecinos Ponderados (WKNN)": KNeighborsClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
    "Bagging Classifier": BaggingClassifier(),
    "Histogram-Based Gradient Boosting": HistGradientBoostingClassifier(),
    "Gaussian Process Classifier": GaussianProcessClassifier(),
    "Decision Tree Classifier": DecisionTreeClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "Calibrated Classifier": CalibratedClassifierCV(),
    "Ridge Classifier": RidgeClassifier(),
}

In [30]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

# Crear una lista para almacenar la precisión de cada modelo
precisiones = []

# Iterar sobre los modelos de clasificación
for modelo_nombre in modelos_mapeados.keys():
    # Crear un nuevo modelo para cada iteración
    modelo_clase = modelos_mapeados[modelo_nombre]  # Obtener la clase del modelo
    modelo = modelo_clase  # Crear una instancia del modelo
    
    # Entrenar el clasificador con los datos de entrenamiento
    modelo.fit(X_train, y_train)
    
    # Hacer predicciones en los datos de validación
    y_pred_val = modelo.predict(X_val)
    
    # Calcular la precisión del modelo en los datos de validación
    precision = accuracy_score(y_val, y_pred_val)
    
    # Calcular el puntaje F1 del modelo en los datos de validación
    puntaje_f1 = f1_score(y_val, y_pred_val, average='weighted')  # 'macro' es útil cuando tienes clases desbalanceadas
    
    # Almacenar la precisión y el puntaje F1 del modelo
    precisiones.append((modelo_nombre, precision, puntaje_f1))

    print(f"Precisión del modelo {modelo_nombre}:", precision)
    print(f"Puntaje F1 del modelo {modelo_nombre}:", puntaje_f1)
    
    # Calcular y mostrar la matriz de confusión
    matriz_confusion = confusion_matrix(y_val, y_pred_val)
    print(f"Matriz de confusión para el modelo {modelo_nombre}:")
    print(matriz_confusion)
    print("\n")

Precisión del modelo Regresión Logística: 0.9565217391304348
Puntaje F1 del modelo Regresión Logística: 0.9563871315116436
Matriz de confusión para el modelo Regresión Logística:
[[5 0 0]
 [0 9 0]
 [0 1 8]]


Precisión del modelo K-Nearest Neighbors (KNN): 0.9130434782608695
Puntaje F1 del modelo K-Nearest Neighbors (KNN): 0.9130434782608695
Matriz de confusión para el modelo K-Nearest Neighbors (KNN):
[[5 0 0]
 [0 8 1]
 [0 1 8]]


Precisión del modelo Árboles de Decisión: 0.8695652173913043
Puntaje F1 del modelo Árboles de Decisión: 0.8691613945349306
Matriz de confusión para el modelo Árboles de Decisión:
[[5 0 0]
 [0 8 1]
 [0 2 7]]


Precisión del modelo Random Forest: 0.9130434782608695
Puntaje F1 del modelo Random Forest: 0.9130434782608695
Matriz de confusión para el modelo Random Forest:
[[5 0 0]
 [0 8 1]
 [0 1 8]]


Precisión del modelo Gradient Boosting: 0.9130434782608695
Puntaje F1 del modelo Gradient Boosting: 0.9119565217391304
Matriz de confusión para el modelo Gradient B

C:\Users\josan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Precisión del modelo Clasificador de Vecinos Ponderados (WKNN): 0.9130434782608695
Puntaje F1 del modelo Clasificador de Vecinos Ponderados (WKNN): 0.9130434782608695
Matriz de confusión para el modelo Clasificador de Vecinos Ponderados (WKNN):
[[5 0 0]
 [0 8 1]
 [0 1 8]]


Precisión del modelo Extra Trees: 0.9130434782608695
Puntaje F1 del modelo Extra Trees: 0.9130434782608695
Matriz de confusión para el modelo Extra Trees:
[[5 0 0]
 [0 8 1]
 [0 1 8]]


Precisión del modelo Bagging Classifier: 0.8695652173913043
Puntaje F1 del modelo Bagging Classifier: 0.8691613945349306
Matriz de confusión para el modelo Bagging Classifier:
[[5 0 0]
 [0 8 1]
 [0 2 7]]


Precisión del modelo Histogram-Based Gradient Boosting: 0.8695652173913043
Puntaje F1 del modelo Histogram-Based Gradient Boosting: 0.8691613945349306
Matriz de confusión para el modelo Histogram-Based Gradient Boosting:
[[5 0 0]
 [0 7 2]
 [0 1 8]]


Precisión del modelo Gaussian Process Classifier: 0.9130434782608695
Puntaje F1 del

In [31]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

# Lista para almacenar los resultados de cada modelo
resultados_modelos = []

# Iterar sobre los modelos de clasificación
for modelo_nombre, modelo_clase in modelos_mapeados.items():
    try:
        # Crear e instanciar el modelo
        modelo = modelo_clase

        # Entrenar el clasificador con los datos de entrenamiento
        modelo.fit(X_train, y_train)

        # Hacer predicciones en los datos de validación
        y_pred_val = modelo.predict(X_val)

        # Calcular la precisión y el puntaje F1
        precision = accuracy_score(y_val, y_pred_val)
        puntaje_f1 = f1_score(y_val, y_pred_val, average='weighted')

        # Guardar resultados en la lista
        resultados_modelos.append({
            'modelo': modelo_nombre,
            'precisión': precision,
            'puntaje_f1': puntaje_f1,
            'matriz_confusión': confusion_matrix(y_val, y_pred_val)
        })

    except Exception as e:
        print(f"Error al entrenar o evaluar el modelo {modelo_nombre}: {e}")


C:\Users\josan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [32]:
print(resultados_modelos)

[{'modelo': 'Regresión Logística', 'precisión': 0.9565217391304348, 'puntaje_f1': 0.9563871315116436, 'matriz_confusión': array([[5, 0, 0],
       [0, 9, 0],
       [0, 1, 8]], dtype=int64)}, {'modelo': 'K-Nearest Neighbors (KNN)', 'precisión': 0.9130434782608695, 'puntaje_f1': 0.9130434782608695, 'matriz_confusión': array([[5, 0, 0],
       [0, 8, 1],
       [0, 1, 8]], dtype=int64)}, {'modelo': 'Árboles de Decisión', 'precisión': 0.8695652173913043, 'puntaje_f1': 0.8691613945349306, 'matriz_confusión': array([[5, 0, 0],
       [0, 8, 1],
       [0, 2, 7]], dtype=int64)}, {'modelo': 'Random Forest', 'precisión': 0.9130434782608695, 'puntaje_f1': 0.9130434782608695, 'matriz_confusión': array([[5, 0, 0],
       [0, 8, 1],
       [0, 1, 8]], dtype=int64)}, {'modelo': 'Gradient Boosting', 'precisión': 0.9130434782608695, 'puntaje_f1': 0.9119565217391304, 'matriz_confusión': array([[5, 0, 0],
       [0, 9, 0],
       [0, 2, 7]], dtype=int64)}, {'modelo': 'Máquinas de Vectores de Soporte (S

In [33]:
# Ordenar los modelos por puntaje F1 de mayor a menor
resultados_modelos.sort(key=lambda x: x['puntaje_f1'], reverse=True)

# Seleccionar los tres modelos con mejores puntajes F1
mejores_tres_modelos = resultados_modelos[:3]
modelos_elegidos=[]
# Imprimir los resultados de los tres mejores modelos
for modelo in mejores_tres_modelos:
    print(f"Modelo: {modelo['modelo']}")
    print(f"Precisión: {modelo['precisión']}")
    print(f"Puntaje F1: {modelo['puntaje_f1']}")
    print(f"Matriz de confusión:\n{modelo['matriz_confusión']}\n")
    modelos_elegidos.append(modelo['modelo'])

print(modelos_elegidos)


Modelo: Regresión Logística
Precisión: 0.9565217391304348
Puntaje F1: 0.9563871315116436
Matriz de confusión:
[[5 0 0]
 [0 9 0]
 [0 1 8]]

Modelo: Máquinas de Vectores de Soporte (SVM)
Precisión: 0.9565217391304348
Puntaje F1: 0.9563871315116436
Matriz de confusión:
[[5 0 0]
 [0 9 0]
 [0 1 8]]

Modelo: AdaBoost
Precisión: 0.9565217391304348
Puntaje F1: 0.9563871315116436
Matriz de confusión:
[[5 0 0]
 [0 9 0]
 [0 1 8]]

['Regresión Logística', 'Máquinas de Vectores de Soporte (SVM)', 'AdaBoost']


In [45]:
mejores_modelos = [(nombre, modelos_mapeados[nombre]) for nombre in modelos_elegidos]
print(mejores_modelos)


[('Regresión Logística', LogisticRegression(max_iter=200, multi_class='multinomial')), ('Máquinas de Vectores de Soporte (SVM)', SVC(probability=True)), ('AdaBoost', AdaBoostClassifier())]


In [46]:
from sklearn.ensemble import VotingClassifier

# Crear el ensemble por votación
ensemble = VotingClassifier(estimators=mejores_modelos, voting='soft')
ensemble.fit(X_train, y_train)
print("Ensemble entrenado con éxito.")

# Evaluar el ensemble en el conjunto de prueba
ensemble_score = ensemble.score(X_val, y_val)
print(f"El ensemble tiene una precisión de {ensemble_score:.2f} en el conjunto de prueba.")


C:\Users\josan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Ensemble entrenado con éxito.
El ensemble tiene una precisión de 0.96 en el conjunto de prueba.


In [48]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Crear el VotingClassifier con modelos específicos
ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=200, multi_class='multinomial')),
    ('svm', SVC(probability=True)),  # Asegúrate de habilitar las probabilidades si usas votación 'soft'
    ('dt', DecisionTreeClassifier())
], voting='hard')  # Puedes cambiar a 'hard' si no necesitas estimar probabilidades

# Suponiendo que tienes X_train y y_train
ensemble.fit(X_train, y_train)

# Evaluar el ensemble
print("Precisión en el conjunto de entrenamiento:", ensemble.score(X_train, y_train))
print("Precisión en el conjunto de prueba:", ensemble.score(X_test, y_test))


Precisión en el conjunto de entrenamiento: 0.984251968503937
Precisión en el conjunto de prueba: 0.9565217391304348


In [40]:


# Evaluar el ensemble en el conjunto de prueba
ensemble_score = ensemble.score(X_test, y_test)
print(f"El ensemble tiene una precisión de {ensemble_score:.2f} en el conjunto de test.")


El ensemble tiene una precisión de 0.96 en el conjunto de test.


# Conclusiones

In [ ]:
## A completar